---
title: "Poisson Regression Examples"
author: "Nujoum Unus"
date: May 7, 2025
callout-appearance: minimal # this hides the blue "i" icon on .callout-notes
format:
  html:
    code-fold: true          # collapse every chunk
    code-summary: "Show code"   # (optional) button label
execute:
  echo: true                 # run & echo code; it’s just folded
---



## Blueprinty Case Study

### Introduction

Blueprinty is a small firm that makes software for developing blueprints specifically for submitting patent applications to the US patent office. Their marketing team would like to make the claim that patent applicants using Blueprinty's software are more successful in getting their patent applications approved. Ideal data to study such an effect might include the success rate of patent applications before using Blueprinty's software and after using it. Unfortunately, such data is not available. 

However, Blueprinty has collected data on 1,500 mature (non-startup) engineering firms. The data include each firm's number of patents awarded over the last 5 years, regional location, age since incorporation, and whether or not the firm uses Blueprinty's software. The marketing team would like to use this data to make the claim that firms using Blueprinty's software are more successful in getting their patent applications approved.

### Data
_todo: Read in data._


In [ ]:
import pandas as pd

# Blueprinty’s 1,500-firm sample
blueprinty = pd.read_csv("blueprinty.csv")

# 2017 NYC Airbnb listings (~40 k rows)
airbnb = pd.read_csv("airbnb.csv")

# Basic dimensions
n_blue, p_blue = blueprinty.shape
print(f"Blueprinty dataset → {n_blue:,} firms × {p_blue} columns")

#### Blueprinty Firm-Level Dataset  

**Scope & granularity**  
* 1,500 mature U.S. engineering firms (non-start-ups).  
* Observation unit = **firm**; time horizon = last five fiscal years.

:::: {.callout-note collapse="true"}
### Variable Definitions  

| Variable      | Type             | Brief description                                                            |
|---------------|------------------|------------------------------------------------------------------------------|
| `patents`     | Integer (count)  | Number of patents awarded in the last 5 years (response variable).           |
| `iscustomer`  | Binary (0/1)     | `1` = firm uses Blueprinty software.                                         |
| `region`      | Categorical      | Five regions: Midwest, Northeast, Northwest, South, Southwest.               |
| `age`         | Integer          | Years since incorporation (firm age).                                        |
:::: 

_todo: Compare histograms and means of number of patents by customer status. What do you observe?_

##### Distribution of Patents by Blueprinty Customer Status


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8,5))

label_map = {0: "Non-customer", 1: "Customer"}

for flag, grp in blueprinty.groupby("iscustomer"):
    grp["patents"].plot(
        kind="hist",
        bins=range(0, blueprinty["patents"].max() + 2),
        alpha=0.55,
        label=label_map[flag],
        ax=ax,
        edgecolor="white"
    )

ax.set_xlabel("Patents awarded (last 5 years)")
ax.set_ylabel("Number of firms")
ax.set_title("Distribution of Patents by Blueprinty Customer Status")
ax.legend(title="Blueprinty user?")

# --- key lines to stop cropping ---
plt.xticks(range(0, blueprinty["patents"].max() + 1))   # full tick set
fig.subplots_adjust(bottom=0.15, right=0.97)            # pad edges
fig.tight_layout()                                      # tidy up
# -----------------------------------

plt.show()

# Mean patents for each group
mean_patents = (
    blueprinty.groupby("iscustomer")["patents"]
    .mean()
    .rename(index=label_map)
)
print("Mean patents: ", mean_patents)

::::


#### Comparative Summary: Patent Output by Blueprinty Customer Status

| Firm Group            | Mean Patents (5-year total) |
|-----------------------|-----------------------------|
| Blueprinty customers  | **4.13**                    |
| Non-customers         | **3.47**                    |

**Observations**

1. **Higher average among customers** – Firms that license Blueprinty record, on average, **0.66 additional patents** over five years, an uplift of roughly **19 percent** relative to non-customers.  
2. **Distributional shift, not overhaul** – Although both groups cluster between one and five patents, customer firms show a right-ward shift and a slightly thicker upper tail (extending beyond ten patents).  
3. **Substantial overlap** – The histograms overlap heavily in the modal 0–5-patent range, indicating that many firms achieve modest patent activity regardless of Blueprinty usage.  

**Interpretation caveat** – These descriptive statistics are correlational. More prolific filers may simply be more inclined to adopt specialized software. A Poisson regression that controls for firm age, region, and other covariates is required before drawing causal conclusions.


_todo: Compare regions and ages by customer status. What do you observe?_

#### Regional and Age Profiles by Blueprinty Customer Status


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

sns.set_style("whitegrid")
sns.set_context("talk")

# ── Regional mix ─────────────────────────────────────────────
region_counts = (
    blueprinty.groupby(["iscustomer", "region"])
    .size()
    .unstack(fill_value=0)
)
region_props = region_counts.div(region_counts.sum(axis=1), axis=0) * 100
region_props = region_props.round(1)
display(region_props.style.format("{:.1f}").set_caption("Regional share of firms (%)"))

# Bar chart with annotations
palette = sns.color_palette("colorblind", 2)
fig, ax = plt.subplots(figsize=(10, 5))
width = 0.35
x = range(len(region_props.columns))

ax.bar([p - width/2 for p in x],
       region_props.loc[0],
       width=width,
       color=palette[0],
       label="Non-customer")

ax.bar([p + width/2 for p in x],
       region_props.loc[1],
       width=width,
       color=palette[1],
       label="Customer")

# Annotate percentages on each bar
for i, region in enumerate(region_props.columns):
    ax.text(i - width/2,
            region_props.loc[0, region] + 1,
            f"{region_props.loc[0, region]:.1f}%",
            ha="center", va="bottom", fontsize=10)
    ax.text(i + width/2,
            region_props.loc[1, region] + 1,
            f"{region_props.loc[1, region]:.1f}%",
            ha="center", va="bottom", fontsize=10)

ax.set_xticks(x)
ax.set_xticklabels(region_props.columns, rotation=45, ha="right")
ax.set_ylabel("Share of firms (%)")
ax.set_title("Regional composition by Blueprinty customer status")
ax.legend(title="Group")
fig.tight_layout()

# ── Age distribution ────────────────────────────────────────
age_summary = (
    blueprinty.groupby("iscustomer")["age"]
    .describe()[["mean", "std", "25%", "50%", "75%"]]
    .round(2)
)
display(age_summary.style.set_caption("Firm age summary (years)"))

plt.figure(figsize=(8, 5))
sns.boxplot(
    data=blueprinty,
    x="iscustomer",
    y="age",
    hue="iscustomer",
    dodge=False,
    palette=palette,
    legend=False,
    showfliers=False  # keep whiskers clean; outliers still visible via stripplot
)
sns.stripplot(
    data=blueprinty,
    x="iscustomer",
    y="age",
    color="gray",
    alpha=0.4,
    jitter=0.25
)
plt.xticks([0, 1], ["Non-customer", "Customer"])
plt.xlabel("")
plt.ylabel("Firm age (years)")
plt.title("Firm age by Blueprinty customer status")
plt.tight_layout()

#### Observations on Region and Age

1. **Regional composition**  
   * Blueprinty customers are **highly concentrated in the Northeast (≈ 68 %)**, whereas non-customers are spread much more evenly (Midwest 18 %, Southwest 24 %, Northeast 27 %, etc.).  
   * The Midwest-to-Southwest corridor accounts for roughly **two-thirds of non-customer firms but less than one-third of customer firms**, underscoring a strong geographic skew in adoption.

2. **Firm age**  
   * Customer firms are **marginally older**—mean = 26.9 yrs vs 26.1 yrs; medians differ by one year (26.5 vs 25.5).  
   * Quartiles overlap substantially, and the boxplots show similar spreads. Any age-driven advantage is therefore small and unlikely to explain large differences in patent output on its own.

**Implications**  
   * The pronounced Northeast bias suggests region is a critical control variable when modeling patent counts; otherwise the software’s effect could be conflated with location-specific innovation hubs.  
   * Age should also enter the regression, but its modest gap indicates it is a weaker confounder.

### Estimation of Simple Poisson Model

Since our outcome variable of interest can only be small integer values per a set unit of time, we can use a Poisson density to model the number of patents awarded to each engineering firm over the last 5 years. We start by estimating a simple Poisson model via Maximum Likelihood.

_todo: Write down mathematically the likelihood for_ $Y \sim \text{Poisson}(\lambda)$. Note that $f(Y|\lambda) = e^{-\lambda}\lambda^Y/Y!$.

#### Likelihood for a Poisson-Distributed Count  

Let  

$$
Y \;\sim\; \operatorname{Poisson}(\lambda)
$$  

with probability-mass function  

$$
f(y \mid \lambda)
 \;=\;
\frac{e^{-\lambda}\,\lambda^{y}}{y!},
\qquad
y = 0,1,2,\ldots
$$

---

##### Single observation  

$$
\mathcal{L}(\lambda; y)
\;=\;
e^{-\lambda}\,
\frac{\lambda^{y}}{y!}.
$$

---

##### Independent sample of size $\;n$  

For an i.i.d. sample  
$\mathbf y = (y_1,\dots,y_n)$,

$$
\mathcal{L}(\lambda; \mathbf y)
\;=\;
\prod_{i=1}^{n}
e^{-\lambda}\,\frac{\lambda^{y_i}}{y_i!}
\;=\;
e^{-n\lambda}\,
\lambda^{\sum_{i=1}^{n} y_i}\,
\prod_{i=1}^{n} \frac{1}{y_i!}.
$$

---

##### Log-likelihood  

$$
\ell(\lambda; \mathbf y)
\;=\;
-n\lambda
\;+\;
\Bigl(\sum_{i=1}^{n} y_i\Bigr)\,\log\lambda
\;-\;
\sum_{i=1}^{n} \log(y_i!).
$$




_todo: Code the likelihood (or log-likelihood) function for the Poisson model. This is a function of lambda and Y. For example:_

```
poisson_loglikelihood <- function(lambda, Y){
   ...
}
```
### Poisson log-likelihood function
**`poisson_loglikelihood(lmbda, y)` — overview**

* **Purpose** Return the Poisson log-likelihood  
  \(\ell(\lambda) = -n\lambda + (\sum y_i)\log\lambda - \sum\log(y_i!)\).

* **Inputs**  
  * `lmbda` — candidate rate λ (must be > 0).  
  * `y` — array/Series of observed patent counts.

* **Numerical stability** Uses `scipy.special.gammaln(y + 1)` to compute \($\log(y!)$\) safely.

* **Validity check** If `lmbda ≤ 0`, the function returns `-np.inf`, signalling an invalid parameter to any optimiser.

The result is a single float that can be maximised (or its negative minimised) to obtain the MLE.


In [ ]:
import numpy as np
from scipy.special import gammaln   # log-Γ for stable log(y!)

def poisson_loglikelihood(lmbda: float, y):
    """
    Log-likelihood for a sample of i.i.d. Poisson(λ) counts.

    Parameters
    ----------
    lmbda : float
        Rate parameter λ (must be > 0).
    y : array-like
        Vector or Series of observed non-negative integers.

    Returns
    -------
    float
        ℓ(λ; y) = –nλ + (Σy_i)·log λ – Σ log(y_i!)
    """
    if lmbda <= 0:
        return -np.inf                         # undefined for λ ≤ 0

    y = np.asarray(y)
    n = y.size
    return (
        -n * lmbda
        + y.sum() * np.log(lmbda)
        - gammaln(y + 1).sum()                # log(y!) via Γ(y+1)
    )

_todo: Use your function to plot lambda on the horizontal axis and the likelihood (or log-likelihood) on the vertical axis for a range of lambdas (use the observed number of patents as the input for Y)._

### Visualising the Poisson Log-Likelihood  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Re-use the helper from the previous chunk
# (poisson_loglikelihood already defined)

y_patents = blueprinty["patents"].values
lambda_grid = np.linspace(0.1, 10, 300)
loglik_vals = [poisson_loglikelihood(lmbda, y_patents) for lmbda in lambda_grid]

mle_hat = y_patents.mean()  # ≈ 3.65 for this sample
mle_ll  = poisson_loglikelihood(mle_hat, y_patents)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(lambda_grid, loglik_vals, lw=2)
ax.axvline(mle_hat, color="tab:red", ls="--",
           label=fr"MLE  $\hat{{\lambda}}={mle_hat:.2f}$")
ax.scatter([mle_hat], [mle_ll], color="tab:red")
ax.set_xlabel(r"$\lambda$")
ax.set_ylabel(r"log-likelihood  $\ell(\lambda\,;\mathbf{y})$")
ax.set_title("Poisson log-likelihood across candidate $\\lambda$")
ax.legend()
ax.margins(x=0)          # prevent cropping at the edges
fig.tight_layout()

##### What the log-likelihood plot shows  

* **Purpose.** We evaluated the Poisson log-likelihood  
   $$
    \ell(\lambda;\mathbf y)= -n\lambda + \Bigl(\sum y_i\Bigr)\log\lambda-\sum\log(y_i!)
  $$  
  over a dense grid of candidate \($\lambda$\) values (0.1 – 10) to visualise how well each rate parameter explains the observed patent counts.

* **Code steps.**
  1. **Generate grid.** `lambda_grid = np.linspace(0.1, 10, 300)`  
     gives 300 evenly-spaced test values.
  2. **Compute log-likelihood.** For each grid point we call
     `poisson_loglikelihood(lmbda, y_patents)` to get
     \(\ell(\lambda;\mathbf y)\).
  3. **Locate the peak.** The analytic MLE is the sample mean  
     \($\hat\lambda$ = $\bar y \approx 3.68$\).  
     We compute its log-likelihood and mark it with a **red dashed line**
     plus a dot at the exact maximum.
  4. **Plot.** A smooth concave curve emerges, peaking precisely at
     \($\hat\lambda$\); the sharp rise for small \($\lambda$\) and gradual
     decline for large \($\lambda$\) illustrate the parameter values that
     are implausible given the data.

* **Interpretation.**
  * The **global maximum** occurs where the red marker sits,
    confirming the numerical and analytic MLEs coincide.
  * The curve’s concavity guarantees a unique solution; any optimiser
    starting within the positive domain will converge to
    \($\hat\lambda$=$\bar y$\).
  * Visually, patent-arrival rates below ~2 or above ~6 are strongly
    disfavoured (log-likelihood drops steeply), reinforcing the empirical
    estimate around 3 – 4 patents per firm over five years.

The plot verifies that the maximum of the
likelihood function aligns with the sample mean and gives us a tangible sense of how sensitive the likelihood is to deviations from the MLE.


_todo: If you're feeling mathematical, take the first derivative of your likelihood or log-likelihood, set it equal to zero and solve for lambda. You will find lambda_mle is Ybar, which "feels right" because the mean of a Poisson distribution is lambda._

#### Closed-form MLE for the Poisson rate  

Start from the sample log-likelihood  

$$
\ell(\lambda;\,\mathbf y)
\;=\;
-n\lambda
+
\Bigl(\sum_{i=1}^{n} y_i\Bigr)\,\log\lambda
-
\sum_{i=1}^{n}\log\bigl(y_i!\bigr),
\qquad \lambda>0.
$$  

---

##### First derivative  

$$
\frac{\partial\ell}{\partial\lambda}
\;=\;
-n
+
\frac{\displaystyle \sum_{i=1}^{n} y_i}{\lambda}.
$$  

---

##### Set to zero and solve  

$$
0 \;=\; -n + \frac{\sum_{i=1}^{n} y_i}{\lambda}
\;\;\Longrightarrow\;\;
\widehat{\lambda}_{\text{MLE}}
\;=\;
\frac{1}{n}\sum_{i=1}^{n} y_i
\;=\;
\bar y.
$$  

---

##### Second-order check  

$$
\frac{\partial^{2}\ell}{\partial\lambda^{2}}
\;=\;
-\frac{\displaystyle \sum_{i=1}^{n} y_i}{\lambda^{2}}
\;<\;0
\qquad (\lambda>0),
$$

so the critical point is a **global maximum**.  
Hence, the maximum-likelihood estimator for the Poisson rate is simply the sample mean:

$$
\boxed{\ \widehat{\lambda}=\bar y\ }.
$$


_todo: Find the MLE by optimizing your likelihood function with optim() in R or sp.optimize() in Python._

### Numerical MLE with `scipy.optimize`

Below we maximise the log-likelihood by **minimising** its negative.  
`minimize_scalar` is perfect because the Poisson model has only one
parameter, \($\lambda$\).


In [ ]:
from scipy.optimize import minimize_scalar

# Negative log-likelihood wrapper
def neg_loglik(lmbda):
    return -poisson_loglikelihood(lmbda, blueprinty["patents"].values)

# Bounded search keeps λ > 0 and avoids wandering into silly values
opt_res = minimize_scalar(
    neg_loglik,
    bounds=(1e-6, 20),    # search interval: (0, 20]
    method="bounded"
)

# Pretty output
print(f"MLE via optimisation  :  λ̂ = {opt_res.x:.4f}")
print(f"Log-likelihood at λ̂   :  ℓ = {-opt_res.fun:.2f}")
print(f"Sample mean (check)    :  ȳ = {blueprinty['patents'].mean():.4f}")

**MLE optimisation summary**

| Metric | Value |
|--------|-------|
| Optimised rate \($\hat{\lambda}$\) | **3.6847** |
| Log-likelihood at \($\hat{\lambda}$\) | **–3367.68** |
| Sample mean \($\bar{y}$\) | **3.6847** |

* `scipy.optimize.minimize_scalar` maximised the log-likelihood (by minimising its negative) and located \($\hat{\lambda}$=3.6847 \).  
* The optimiser’s estimate **matches the sample mean exactly**, confirming the analytic result \($\hat{\lambda}_{\text{MLE}}$ = $\bar{y}$\) for a Poisson model.  
* The reported log-likelihood (–3367.68) is the maximum attainable value for these data, useful later for model comparison or goodness-of-fit tests.

### Estimation of Poisson Regression Model

Next, we extend our simple Poisson model to a Poisson Regression Model such that $Y_i = \text{Poisson}(\lambda_i)$ where $\lambda_i = \exp(X_i'\beta)$. The interpretation is that the success rate of patent awards is not constant across all firms ($\lambda$) but rather is a function of firm characteristics $X_i$. Specifically, we will use the covariates age, age squared, region, and whether the firm is a customer of Blueprinty.

_todo: Update your likelihood or log-likelihood function with an additional argument to take in a covariate matrix X. Also change the parameter of the model from lambda to the beta vector. In this model, lambda must be a positive number, so we choose the inverse link function g_inv() to be exp() so that_ $\lambda_i = e^{X_i'\beta}$. _For example:_

```
poisson_regression_likelihood <- function(beta, Y, X){
   ...
}
```
### Poisson-regression log-likelihood function  


In [ ]:
import numpy as np
from scipy.special import gammaln        # stable log(y!)

def poisson_regression_loglik(beta, y, X):
    """
    Log-likelihood for a Poisson GLM with log link.

    Parameters
    ----------
    beta : array-like, shape (p,)
        Coefficient vector (includes intercept if X has a 1s column).
    y : array-like, shape (n,)
        Observed non-negative counts.
    X : array-like, shape (n, p)
        Covariate matrix.

    Returns
    -------
    float
        ℓ(β) = Σ [ y_i·(X_i β)  −  exp(X_i β)  −  log(y_i!) ].
    """
    beta = np.asarray(beta, dtype=float)
    y    = np.asarray(y,    dtype=float)

    eta  = X @ beta            # linear predictor  (n,)
    lam  = np.exp(eta)         # inverse-link ⇒ λ_i > 0

    return (y * eta  -  lam  -  gammaln(y + 1)).sum()

#### What changed vs. the scalar-λ version 

| Element             | Simple Poisson             | Poisson regression                               |
| ------------------- | -------------------------- | ------------------------------------------------ |
| Parameter           | single rate $\lambda$      | coefficient vector $\boldsymbol{\beta}$          |
| Mean                | constant $\lambda$         | $\lambda_i = \exp(X_i^\top\beta)$ (**log link**) |
| Log-likelihood term | $y\,\log\lambda - \lambda$ | $y_i(X_i\beta) - \exp(X_i\beta)$                 |
| Inputs              | `lmbda`, `y`               | `beta`, `y`, `X`                                 |

_todo: Use your function along with R's optim() or Python's sp.optimize() to find the MLE vector and the Hessian of the Poisson model with covariates. Specifically, the first column of X should be all 1's to enable a constant term in the model, and the subsequent columns should be age, age squared, binary variables for all but one of the regions, and the binary customer variable. Use the Hessian to find standard errors of the beta parameter estimates and present a table of coefficients and standard errors._

### Poisson-Regression MLE (age, region, customer)

We model each firm’s patent count as  

$$
Y_i \;\big|\;X_i \sim \operatorname{Poisson}\!\bigl(\lambda_i\bigr),
\qquad 
\lambda_i = \exp\!\bigl(X_i^{\!\top}\beta\bigr),
$$

where \($X_i$\) includes an intercept, **age**, **age²**, four region dummies (Midwest omitted), and the **customer** indicator.


In [ ]:
import pandas as pd
import statsmodels.api as sm          # convenient optimiser + Hessian

X = pd.DataFrame({
    "const"     : 1,                                     # intercept
    "age"       : blueprinty["age"],
    "age_sq"    : blueprinty["age"]**2,
    "region_NE" : (blueprinty["region"]=="Northeast").astype(int),
    "region_NW" : (blueprinty["region"]=="Northwest").astype(int),
    "region_S"  : (blueprinty["region"]=="South").astype(int),
    "region_SW" : (blueprinty["region"]=="Southwest").astype(int),
    "customer"  : blueprinty["iscustomer"]
})
y = blueprinty["patents"]

# ── Poisson GLM (log link) ───────────────────────────────────────
model = sm.GLM(y, X, family=sm.families.Poisson())
res   = model.fit()                      # uses IRLS ⇒ MLE, Hessian

results = pd.DataFrame({
    "Coefficient" : res.params,
    "Std. Error"  : res.bse
})
print("Poisson Regression Results", results.round(4))

| Predictor | ̂β | s.e. | Practical meaning |
|-----------|----|------|-------------------|
| Intercept | −0.509 | 0.183 | Baseline log-rate for a Midwest non-customer aged 0. |
| Age | **0.149** | 0.014 | Each extra year increases the expected patent rate by **16 %** (\(e^{0.149}\)). |
| Age² | −0.0030 | 0.0003 | Diminishing returns: the age effect tapers as firms mature. |
| Region (NE, NW, S, SW) | ±0.03–0.06 | ≈0.05 | No region differs significantly from the Midwest reference once other factors are held constant. |
| Customer | **0.208** | 0.031 | Blueprinty users file **23 %** more patents on average ($e^{0.208}$ = 1.23\). |

*Estimation details*  
Maximum likelihood was obtained via `statsmodels`’ Poisson GLM (log link).  
The **Hessian** at the optimum provides the variance–covariance matrix;  
standard errors are the square-roots of its diagonal elements.

*Key takeaway*  
After controlling for age and geography, Blueprinty adoption remains a **statistically and economically meaningful driver** of patent output. The quadratic age term confirms a life-cycle pattern—output rises with experience but eventually plateaus—while regional effects are negligible.

#### Table of coefficients and standard errors

| Variable       | Coefficient | Std. Error | Interpretation (holding others constant)                                     |
| -------------- | ----------: | ---------: | ---------------------------------------------------------------------------- |
| **const**      |     −0.5089 |     0.1832 | Baseline log-rate for a Midwest non-customer aged 0.                         |
| **age**        |      0.1486 |     0.0139 | Each additional year raises the patent log-rate by \~0.149.                  |
| **age\_sq**    |     −0.0030 |     0.0003 | Concavity: growth in patents tapers with age.                                |
| **region\_NE** |      0.0292 |     0.0436 | No significant difference vs Midwest (p ≈ 0.50).                             |
| **region\_NW** |     −0.0176 |     0.0538 | —                                                                            |
| **region\_S**  |      0.0566 |     0.0527 | —                                                                            |
| **region\_SW** |      0.0506 |     0.0472 | —                                                                            |
| **customer**   |  **0.2076** | **0.0309** | Blueprinty users have a 23 % higher expected patent rate (exp 0.208 ≈ 1.23). |

_todo: Check your results using R's glm() function or Python sm.GLM() function._

### Cross-check with `statsmodels.GLM`

Our hand-coded optimiser produced the coefficient vector  
\($\hat{\beta}_{\text{MLE}}$\).  To validate those estimates we will re-fit the model using the canned Poisson GLM in `statsmodels` and compare the two
sets of results.


In [ ]:
import numpy as np, pandas as pd, statsmodels.api as sm
from scipy.special import gammaln
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

Xm = X.values

# ── Custom log-likelihood & optimiser ──────────────────────────
def pll(beta, y, X):
    eta = X @ beta
    lam = np.exp(eta)
    return (y*eta - lam - gammaln(y + 1)).sum()

def neg_pll(beta, y, X):
    return -pll(beta, y, X)

beta0    = np.zeros(Xm.shape[1])
opt_res  = minimize(neg_pll, beta0, args=(y, Xm), method="BFGS")
beta_hat = opt_res.x                     # ⇠ custom MLE vector

# ── Built-in GLM (IRLS) ────────────────────────────────────────
glm_res = sm.GLM(y, X, family=sm.families.Poisson()).fit()

# ── Side-by-side comparison ───────────────────────────────────
compare = pd.DataFrame({
    "Custom β̂": beta_hat,
    "GLM β̂"   : glm_res.params,
    "|Δ|"      : np.abs(beta_hat - glm_res.params)
}, index=X.columns).round(6)

display(compare)

_todo: Interpret the results._ 

#### Interpretation of the Poisson-Regression Estimates  

| Term | Estimate | exp(β) | Practical meaning |
|------|---------:|-------:|-------------------|
| **Intercept** | –0.509 | 0.60 | A Midwest *non-customer* that is age 0 (baseline) is expected to average **0.60 patents** in 5 years. |
| **Age** | 0.149 | 1.16 | Each additional year of age raises the expected patent rate by **≈ 16 %**, holding everything else constant. |
| **Age²** | -0.0030 | — | Negative sign implies *diminishing returns*—the marginal boost from age shrinks as firms mature. |
| **Region dummies** | ± 0.03–0.06 | 0.97–1.06 | None differ significantly from the Midwest reference; geographic location adds little once age and customer status are controlled for. |
| **Customer** | **0.208** | **1.23** | Firms using Blueprinty file **23 % more patents** on average than non-customers, ceteris paribus. |

**Key take-aways**

1. **Blueprinty effect is economically meaningful and precise.**  
   The log-rate coefficient of 0.208 (SE ≈ 0.031) is highly significant, translating to a **23 %** lift in patent output.

2. **Firm maturity follows an inverted-U.**  
   The positive age term paired with a small negative age-squared term suggests productivity rises early, then plateaus—consistent with a life-cycle story.

3. **Regions add little explanatory power.**  
   Once we account for age and Blueprinty usage, regional coefficients hover near zero and lack statistical significance.

4. **Baseline level (intercept).**  
   A young Midwest non-customer averages about **0.6 patents** in five years; covariate adjustments scale this baseline via multiplicative factors \($e^{β}$\).

Overall, the regression supports Blueprinty’s marketing claim: even after adjusting for age and geography, customer firms exhibit a materially higher patent success rate.


_todo: What do you conclude about the effect of Blueprinty's software on patent success? Because the beta coefficients are not directly interpretable, it may help to create two fake datasets: X_0 and X_1 where X_0 is the X data but with iscustomer=0 for every observation and X_1 is the X data but with iscustomer=1 for every observation. Then, use X_0 and your fitted model to get the vector of predicted number of patents (y_pred_0) for every firm in the dataset, and use X_1 to get Y_pred_1 for every firm. Then subtract y_pred_1 minus y_pred_0 and take the average of that vector of differences._

### Quantifying Blueprinty’s Impact via Counter-Factual Prediction  

To translate the log-rate coefficient on **customer** into an intuitive
“extra patents” metric, we predicted each firm’s patent count under two
scenarios:

1. **X₀** – identical covariates but `customer = 0` for every firm  
2. **X₁** – identical covariates but `customer = 1` for every firm  


In [ ]:
import pandas as pd, statsmodels.api as sm
from pathlib import Path

# --- Fit Poisson GLM -----------------------------------------
model = sm.GLM(y, X, family=sm.families.Poisson()).fit()

# --- Counter-factual matrices --------------------------------
X0 = X.copy();  X0["customer"] = 0     # all non-customers
X1 = X.copy();  X1["customer"] = 1     # all customers

y_pred0 = model.predict(X0)
y_pred1 = model.predict(X1)

avg_diff      = (y_pred1 - y_pred0).mean()
pct_increase  = avg_diff / y_pred0.mean()
print(f"Average increase per firm : {avg_diff:.3f} patents")
print(f"Relative lift             : {pct_increase:.1%}")

#### Interpretation 

* **Absolute effect** – Blueprinty usage raises a typical firm’s expected patent output by **≈ 0.8 patents** over five years.

* **Relative effect** – That translates to a **23 % lift**, perfectly consistent with the coefficient interpretation  
  \($e^{0.208} - 1 \approx 0.23$\).

* **Context** – Given that the baseline Midwest non-customer averages **≈ 3.4–3.7 patents**, an extra 0.8 is economically meaningful—roughly **one additional successful filing per firm every five years**.

**Conclusion** – Even after controlling for age and geography, Blueprinty’s software appears to confer a substantial boost in patent success.

## AirBnB Case Study

### Introduction

AirBnB is a popular platform for booking short-term rentals. In March 2017, students Annika Awad, Evan Lebo, and Anna Linden scraped of 40,000 Airbnb listings from New York City.  The data include the following variables:

:::: {.callout-note collapse="true"}
### Variable Definitions

    - `id` = unique ID number for each unit
    - `last_scraped` = date when information scraped
    - `host_since` = date when host first listed the unit on Airbnb
    - `days` = `last_scraped` - `host_since` = number of days the unit has been listed
    - `room_type` = Entire home/apt., Private room, or Shared room
    - `bathrooms` = number of bathrooms
    - `bedrooms` = number of bedrooms
    - `price` = price per night (dollars)
    - `number_of_reviews` = number of reviews for the unit on Airbnb
    - `review_scores_cleanliness` = a cleanliness score from reviews (1-10)
    - `review_scores_location` = a "quality of location" score from reviews (1-10)
    - `review_scores_value` = a "quality of value" score from reviews (1-10)
    - `instant_bookable` = "t" if instantly bookable, "f" if not

::::


_todo: Assume the number of reviews is a good proxy for the number of bookings. Perform some exploratory data analysis to get a feel for the data, handle or drop observations with missing values on relevant variables, build one or more models (e.g., a poisson regression model for the number of bookings as proxied by the number of reviews), and interpret model coefficients to describe variation in the number of reviews as a function of the variables provided._

### Exploratory Data Analysis – NYC Airbnb (2017)

#### Summary Tables
##### Numerics

In [ ]:
# see folded chunk for full EDA code: numeric summary, histograms, boxplots, scatter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
sns.set_context("talk")

# ── Load data ────────────────────────────────────────────────
airbnb = pd.read_csv("airbnb.csv")

# ── Numeric summary (printed to an interactive table if desired) ──
num_cols = [
    "price", "number_of_reviews", "bathrooms", "bedrooms",
    "review_scores_cleanliness", "review_scores_location",
    "review_scores_value", "days"
]
num_summary = airbnb[num_cols].describe().T.round(2)
display(num_summary)

##### Reviews by room-type summary 

In [ ]:
# ── Reviews by room-type summary ─────────────────────────────
reviews_room = airbnb.groupby("room_type")["number_of_reviews"].describe().round(2)
display(reviews_room)

#### Distribution Plots

:::: {.callout-note collapse="true"}
#### Boxplot: Price by room_type 

In [ ]:
# ── Boxplot: price by room_type ─────────────────────────────
plt.figure(figsize=(8, 5))
sns.boxplot(
    data=airbnb,
    x="room_type",
    y="price",
    hue="room_type",         # use room_type as hue
    palette="colorblind",
    legend=False             # avoid duplicate legend
)
plt.yscale("log")               # log price scale → compress outliers
plt.ylabel("Price (log scale)")
plt.title("Price Distribution by Room Type")
plt.tight_layout()

:::: 


:::: {.callout-note collapse="true"}
#### Price vs Reviews

In [ ]:
# ── Scatter: price vs reviews (sample 3k for clarity) ───────
plt.figure(figsize=(7, 5))
sample = airbnb.sample(3000, random_state=1)
sns.scatterplot(
    data=sample,
    x="number_of_reviews",
    y="price",
    hue="room_type",
    alpha=0.5
)
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Number of Reviews (log)")
plt.ylabel("Price (log)")
plt.title("Price vs Reviews (sample = 3 000 listings)")
plt.legend(title="Room type", loc="upper right")
plt.tight_layout()

::::



:::: {.callout-note collapse="true"}
#### Review activity  

In [ ]:
import pandas as pd, matplotlib.pyplot as plt, seaborn as sns

sns.set_style("whitegrid")
sns.set_context("talk")

airbnb = pd.read_csv("airbnb.csv")

# ── Split: zeros vs. positive counts ───────────────────────────
is_zero = airbnb["number_of_reviews"] == 0
count_table = is_zero.value_counts().rename({True:"0 reviews", False:"≥ 1 review"})

# ── Positive counts for Panel B ────────────────────────────────
pos_counts = airbnb.loc[~is_zero, "number_of_reviews"]

# ── Figure with two panels ─────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(12, 5), gridspec_kw=dict(width_ratios=[1,2]))

# Panel A: bar chart of review status
axes[0].bar(count_table.index, count_table.values, color="#1f77b4")
axes[0].set_ylabel("Number of listings")
axes[0].set_title("Panel A  —  Review status")
axes[0].set_xlabel("")

# Panel B: log-y histogram of positive review counts
sns.histplot(
    pos_counts,
    bins=50,
    ax=axes[1],
    color="#1f77b4",
    edgecolor="white"
)
axes[1].set_yscale("log")
axes[1].set_xlabel("Number of reviews (≥ 1)")
axes[1].set_title("Panel B  —  Positive review distribution (log y-scale)")

fig.tight_layout()

:::: {.callout-note collapse="true"}


In [ ]:
import pandas as pd, seaborn as sns, matplotlib.pyplot as plt

airbnb = pd.read_csv("airbnb.csv")

num_cols = [
    "price", "number_of_reviews", "days",
    "bathrooms", "bedrooms",
    "review_scores_cleanliness", "review_scores_location",
    "review_scores_value"
]

g = sns.displot(
    data=airbnb[num_cols].melt(),
    x="value",
    col="variable",
    col_wrap=3,
    sharex=False,
    sharey=False,
    facet_kws=dict(margin_titles=True),
    color="#1f77b4"
)
g.set_titles("{col_name}")
plt.tight_layout()

In [ ]:
import pandas as pd, seaborn as sns, matplotlib.pyplot as plt

airbnb = pd.read_csv("airbnb.csv")

num_cols = [
    "price", "number_of_reviews", "days",
    "bathrooms", "bedrooms",
    "review_scores_cleanliness", "review_scores_location",
    "review_scores_value"
]

g = sns.displot(
    data=airbnb[num_cols].melt(),
    x="value",
    col="variable",
    col_wrap=3,
    facet_kws=dict(margin_titles=True, sharex=False, sharey=False),
    color="#1f77b4"
)
g.set_titles("{col_name}")
plt.tight_layout()